# CircuitSEQ API examples

In [1]:
import requests
import os
import pandas as pd

In [19]:
rest_api_url = "https://circuitseq.iwr.uni-heidelberg.de/api"

## Public API

This can be accessed by anyone without requiring an authorization token.

### `/remaining`

- returns how many samples are still available this week
- doesn't require an authentication token

In [10]:
# requests is the standard Python library for this:
response = requests.get(f"{rest_api_url}/remaining")

/home/lkeegan/.pyenv/versions/3.9.4/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
# status code is 200 on success, 401 or other if something went wrong:
response.status_code

200

In [12]:
# the request returns data in json format (which is converted to a Python dict by requests):
response.json()

{'message': '', 'remaining': 90}

## Admin API

An authorization token for an admin account is required

In [ ]:
# if we try to access an admin endpoint without suitable authentication we get an error:
response = requests.get(f"{rest_api_url}/admin/samples")

In [ ]:
response.status_code

In [ ]:
response.json()

## Auth token
- you can get create an API token from the admin page
- then export it as an evironment variable on your computer e.g. `export CIRCUITSEQ_API_TOKEN=abca12421...`
- need to add this token to your header when making requests if you want to authenticate yourself
- NOTE: currently these tokens are invalidated whenever the code is updated & the server is restarted (which is currently quite often!)
  - if this happends your previously working authorized request will return a 422 status code with `Signature verification failed` in the json
  - generating a new token from the admin page and exporting it should fix this

In [14]:
# get the API token from the environment variable (better not to directly add it to your script)
auth_token = os.environ["CIRCUITSEQ_API_TOKEN"]
# make an authorization header for requests to use
auth_header = {"Authorization": f"Bearer {auth_token}"}

### `/admin/samples`

- returns current samples and previous samples

In [15]:
response = requests.get(f"{rest_api_url}/admin/samples", headers=auth_header)

/home/lkeegan/.pyenv/versions/3.9.4/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
response.status_code

200

In [17]:
response.json().keys()

dict_keys(['current_samples', 'previous_samples'])

In [18]:
pd.DataFrame(response.json()["current_samples"])

,concentration,date,email,has_results_fasta,has_results_gbk,has_results_zip,id,name,primary_key,reference_sequence_description,running_option
0,200,"Tue, 13 Dec 2022 00:00:00 GMT",liam.keegan@iwr.uni-heidelberg.de,False,False,False,1,asd,22_50_A1,None,afsadf
1,999,"Tue, 13 Dec 2022 00:00:00 GMT",liam.keegan@iwr.uni-heidelberg.de,False,False,False,2,234,22_50_A2,Reference,afsadf
2,800,"Tue, 13 Dec 2022 00:00:00 GMT",liam.keegan@iwr.uni-heidelberg.de,False,False,False,3,sgsdf,22_50_A3,None,afsadf
3,200,"Tue, 13 Dec 2022 00:00:00 GMT",liam.keegan@iwr.uni-heidelberg.de,False,False,False,4,sdfsdf,22_50_A4,None,afsadf
4,202,"Tue, 13 Dec 2022 00:00:00 GMT",liam.keegan@iwr.uni-heidelberg.de,False,False,False,5,fddf,22_50_A5,None,afsadf
5,666,"Tue, 13 Dec 2022 00:00:00 GMT",liam.keegan@iwr.uni-heidelberg.de,False,False,False,6,adsfsdf,22_50_A6,Reference,sdfwfwe


### `/admin/zipsamples`

- download a zipfile with a tsv of the samples table and reference sequence fasta files for this week

In [ ]:
response = requests.post(f"{rest_api_url}/admin/zipsamples", headers=auth_header)

In [ ]:
response.status_code

In [ ]:
# write downloaded binary data to a file:
with open("samples.zip", "wb") as f:
    f.write(response.content)

### `/admin/result`

- upload a zipfile of results for a sample
- assumes the zipfile is named `[PRIMARY_KEY]_[NAME].zip`
- on success returns the path on the server where the zipfile is stored

In [ ]:
with open(
    "../backend/tests/data/results/22_46_A2_ZIP_TEST_pMC_Final_Kan.zip", "rb"
) as f:
    response = requests.post(
        f"{rest_api_url}/admin/result", files={"file": f}, headers=auth_header
    )

In [ ]:
response.status_code

In [ ]:
response.json()